# Background

### What is this about
This is an expirement on the difference between two forms of naive bayes + laplace smoothing in text classification, namely:
1. Multivariate Bernoulli Event Model (MVBEM)
2. Multinomial Event Model (MNEM)

### The Goal 
The goal is to apply both versions of Naive Bayes to predict whether an email text is a SPAM or HAM and measure whether one technique is better than the other and which areas do they both excell at.

### Naive Bayes summary
The Naive Bayes, when applied to text classification, assumes that the words are conditionally independent from each other and is not affect by the classification of the text. This allows us to reduce the computation requires using bayes rule. Thus giving us the following PDF
$$
P(y |x) =\frac{P(y)\prod_{i = 1}^DP(x_i|y)}{\prod_{j=1}^KP(y =j)\prod_{i=1}^DP(x_i|y = j)}
$$

where $D$ is the number of words to consider and $K$ is the number of classifications
### Multivariate Bernoulli event Model
The MVBEM aims to calculate the provability of a certain words (from the list of words we are considering) from occuring after learning that a text is SPAM or HAM. Because the classification is binary, we only need to calculate one of the provabilities to know whether the text is spam or not. This then gives us the following Provability distribution 
$$
P(y =1|x) =\frac{\prod_{i = 1}^DP(x_i|y = 1)P(y = 1)}{\prod_{i=1}^DP(x_i|y = 0)P(y = 0) + \prod_{i=1}^DP(x_i|y = 1)P(y = 1)}
$$

Because a word can only occur or not and a classification can only be SPAM or HAM, we will use the bernoulli distribution to model each, thus its name. This then gives us the following relationships:
$$
P(x_i = 1) = \phi_{i} \\
P(x_i = 0) = 1 - \phi_{i} \\
P(x_i) = \phi_i^{I\{\ x_i = 1\}}(1-\phi_i)^{1 - I\{ x_i = 1\}} \\
P(x_i =1 |y = 1) = \phi_{i|y = 1}^{I\{x_i = 1 \wedge y^{(i)} = 1\}} \\
P(x_i|y = 1) = \phi_{i|y = 1}^{I\{x_i = 1 \wedge y^{(i)} = 1\}}(1 - \phi_{i|y = 1})^{I{\{y^{(i)} = 1\}} - I\{x_i = 1 \wedge y^{(i)} = 1\}} \\ 
P(x_i|y = 0) = \phi_{i|y = 0}^{I\{x_i = 1 \wedge y^{(i)} = 0\}}(1 - \phi_{i|y = 0})^{I\{y^{(i)} = 0\} - I\{x_i = 0 \wedge y^{(i)} = 0\}}
$$
After applying these to the PDF, we can use MLE to get the best parameters. We then apply laplace smoothing to take into account zero possibilities thus giving us the following results:

$$
\phi_y = \frac{\sum_{j=1}^NI\{y^{(j)} = 1\}}{N}\\
\phi_{i|y = 1} = \frac{a +\sum_{j=1}^NI\{x_i^{(j)} = 1 \wedge y^{(j)} = 1\}} {2a +\sum_{j=1}^NI\{y^{(j)} = 1\}} \\
\phi_{i|y = 0} = \frac{a+ \sum_{j=1}^NI\{x_i^{(j)} = 1 \wedge y^{(j)} = 0\}}{2a +\sum_{j=1}^NI\{y^{(j)} = 0\}}
$$

where $a$ is our smoothing parameter

The above calculations can be simplified using matrices
$$
\phi_y = \frac{1}{N}sum(Y) \\
\phi_{i|y=1} = \frac{YX}{sum(Y)} \\
\phi_{i|y=0} = \frac{(1-Y)X}{N - sum(Y)}
$$

where 

$Y \epsilon \R^{1 \times N}$ = where the $i'th$  entry represents whether the $i'th$ text is a spam or not (1, 0)

$X \epsilon\R^{N \times D}$ = where the $i'th$ row is a vector representing whether certain words occured in the $i'th$ text

lastly to make predicitions we will use the following formula:

$$
P(x_i|y=1) = \frac{(1-\phi_y)prod(P_{i|y=0}^{X} \cdot(1-P_{i|y=0})^{1-X})}{\phi_yprod(P_{i|y=1}^{X} \cdot(1-P_{i|y=1})^{1-X}} < 1 : SPAM
$$

where 

$\phi_y \epsilon \R$ 

$P_{i|y=0}\epsilon\R^{1\times D}$ = is a row vector where the $i'th$ entry is $\phi_{i|y=0}$ 

$P_{i|y=1}\epsilon\R^{1\times D}$ = is a row vector where the $i'th$ entry is $\phi_{i|y=1}$ 

$X \epsilon \R^{1 \times D}$ = is the feature vector to predict where the $i'th$ entry represents the existance of the $i'th$  word in the text (either 1 or 0) 

$D$ = denotes the number of words to consider
### Multinomial Event Model
Unlike the MVBEM, the MNEM calculates that provability that a certain word will occur at a certain position in the text. This allows the model to take into account repeating words, which is ignored by the previous model. Because every position can have $D$ possitibilies, we will use the multi-nomial model to model this provability. This then gives us the following PDF

$$
P(y =1|x) =\frac{\prod_{i = 1}^LP(x_i|y = 1)P(y = 1)}{\prod_{i=1}^LP(x_i|y = 0)P(y = 0) + \prod_{i=1}^LP(x_i|y = 1)P(y = 1)}
$$

where $L$ is the length of the text. We will then use the following models
$$
P(y) = \phi_y^{I\{y^{(i)} = 1\}}(1 - \phi_y)^{1 - {I\{y^{(i)} = 1\}}} \\
P(x_k) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i\}}}{\sum_{i=1}^D\phi_i} \\
P(x_k|y=1) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i \wedge y^{(k) = 1}\}}}{\sum_{i=1}^D\phi_i^{I\{ y^{(k) = 1}\}}} \\ 
P(x_k|y=0) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i \wedge y^{(k)} = 0\}}}{\sum_{i=1}^D\phi_i^{I\{y^{(k) = 0}\}}}P(y) = \phi_y^{I\{y^{(i)} = 1\}}(1 - \phi_y)^{1 - {I\{y^{(i)} = 1\}}} \\
P(x_k) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i\}}}{\sum_{i=1}^D\phi_i} \\
P(x_k|y=1) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i \wedge y^{(k) = 1}\}}}{\sum_{i=1}^D\phi_i^{I\{ y^{(k) = 1}\}}} \\ 
P(x_k|y=0) = \frac{\prod_{i=1}^{D}\phi_i^{I\{x_k = i \wedge y^{(k)} = 0\}}}{\sum_{i=1}^D\phi_i^{I\{y^{(k) = 0}\}}}
$$

And it turns out if we compute for each parameter, in addition with laplace smoothing, we will have
$$
\phi_y = \frac{\sum_{i = 1}^NI\{y^{(i)} = 1\}}{N} \\
\phi_{k|y=1}= \frac{1 + \sum_{i = 1}^NI\{y^{(i)} = 1\}\sum_{j=1}^DI\{\ x_j^{(i)}=k\}}{D + \sum_{i = 1}^NI\{y^{(i)} = 1\}L_i} \\
\phi_{k|y=0}= \frac{1 + \sum_{i = 1}^NI\{y^{(i)} = 0\}\sum_{j=1}^DI\{\ x_j^{(i)}=k\}}{D + \sum_{i = 1}^NI\{y^{(i)} = 0\}L_i}
$$

where $\phi_{k|y=1}$ is the provability that a certain word will occur at a certain possition given that the text is a SPAM. Notice that there is not indicator of the position this is because the provability that a ceratain word will occur is not affected by the position, this is also one of the major drawbacks of this approach as position is vital in texts as words follow a certain gramatical structure

Lastly to make predictions, we will use the following formula
$$
\frac{(1 - \phi_y)\sum_{i=1}^D\phi_{i|y=1}}{\phi_y\sum_{i=1}^D\phi_{i|y=0}} \prod_{k=1}^L\frac{\prod_{i=1}^{D}\phi_{i|y=0}^{I\{x_k = i \}}}{\prod_{i=1}^{D}\phi_{i|y=1}^{I\{x_k = i \}}} < 1 : spam
$$

### Training Techniques
Notice that in order for both Naive Bayes to work, we have to consider some set of $D$ words. One way to do this is to just consider the top $K$ words in the dictionary. But this may be bad especially given the fact the new words come up each day. A solution to this is to only consider the words that appeared in the training set. This is the tecnique we will use. The problem with this is that the number of words to consider can balloon exponentially thus exceeding the RAM (this happened to my 24 gb ram laptop). To compensate, a good approach will be to start with the top K words in dictionary then add new words with a certain limit. Once we exceed that limit, we stop adding new words

Also, notice that for multi-nomial event models, when predicting, we have to take into account the fact that the word we are processing may not exist in our set of words $D$. One way to solve this is to assign one number to a token that signifies a word does not exist. Let’s say our words are of length 10,000, we can denote 10,001 as a representation of words that does not exist.

Also, all words are lowercased to take into account similar words with different capitalization. I am aware that the capitalization of words also contribute to determining whether a text is SPAM or HAM, but to lessen the amount of parameters (words) to consider, I will take this gamble. 

# Multivariate Bernoulli Event Model

### STEP 1: DATA PROCESSING
First we have to process the data into our intended format. Specifically to 

$Y \epsilon \R^{1 \times N}$ = where the $i'th$  entry represents whether the $i'th$ text is a spam or not (1, 0)

$X \epsilon\R^{N \times D}$ = where the $i'th$ row is a vector representing whether certain words occured in the $i'th$ text

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import gc

# Configurations
dataset_path1 = "dataset/spam_ham_dataset.csv"
dataset_path2 = "dataset/spam.csv"
chunksize = 1000                            # Adjust depending on your RAM capacity
words_to_consider_limit = 10_000            # Adjust depending on your RAM capacity. this is also same with D 
len_words_to_skip = 15                      # If words exceed this limit, we ignore it
classifications = {"spam": 1, "ham": 0}     # change accordingly

# The convert to int parameter is used for data where the labeling is spam=1 and ham=0 instead of just 1 and 0
def parse_data(chunksize, path, text_col_num, label_col_num, encoding="UTF-8", convert_to_int=False):
    print("Parsing data")
    df = pd.read_csv(path, encoding=encoding, chunksize=chunksize)

    # Things to return
    words_to_consider = {}
    Y = None
    X = None

    chunks_processed = 0
    for chunk in df:
        chunk = chunk.to_numpy()

        if convert_to_int:
            y_chunk = np.array([[classifications[c] for c in chunk[:, label_col_num]]])
        else:
            y_chunk = np.array(chunk[:, label_col_num])
        
        if chunks_processed == 0:                                   
            Y = tf.constant(y_chunk, dtype=tf.uint8)                                                                                            
        else:
            Y = tf.concat((Y, y_chunk), axis=0)
        
        # Parse each text to a matrix
        for i in range(len(chunk)):
            print(f"Processing {chunks_processed * chunksize + i}")
            text = chunk[i, text_col_num]
            matrix = np.array([map_text_to_matrix(text, words_to_consider)])                                                                                                                                                                                       

            if chunks_processed == 0 and i == 0:
                X = tf.constant(matrix, dtype=tf.uint8)                                                
            else:                           
                X = tf.concat((X, matrix), axis=0)

            del text, matrix
            print(f"Successfully processed {chunks_processed * chunksize + i}")

        # Manual deletion to save RAM
        chunks_processed += 1
        del chunk, y_chunk
        gc.collect()

    # Adjust size of every row in x_list to be D
    D = len(words_to_consider)
    N = len(X)
    print("Successfully parsed data")
    return X, Y, N, D, words_to_consider

def map_text_to_matrix(text, words_to_consider):
    D = len(words_to_consider)
    matrix = np.zeros(words_to_consider_limit)

    for word in text.split():
        word = word.lower()
        if word not in words_to_consider:
            if D >= words_to_consider_limit:
                continue
            
            if len(word) >= len_words_to_skip:
                continue

            words_to_consider[word] = D
            D += 1
        
        matrix[words_to_consider[word]] = 1
    
    return matrix

def pad_to_size(arr, size):
    # Assumes that size >= len(arr)
    return np.pad(arr, (0, size - len(arr)), mode='constant')

def display_tensor(tensor, name):
    print(f"{name}: \n", tensor.numpy())

# Parse the data
X, Y, N, D, words_to_consider = parse_data(chunksize, dataset_path2, 2, 3, "latin-1", True)

print("Total number of words to consider: ", D)
print("Total number of emails: ", N)                
print("Words to consider: ", words_to_consider)                  
display_tensor(X, "X")                                                                                                                                                                                                                                                 
display_tensor(Y, "Y")


SyntaxError: unmatched ']' (2809127371.py, line 31)